# Object and scene detection using Amazon Rekognition

***
This notebook provides a walkthrough of [object detection API](https://docs.aws.amazon.com/rekognition/latest/dg/labels.html) in Amazon Rekognition to identify objects.
***

In [ ]:
# Initialize dependencies
import boto3
import botocore
from IPython.display import HTML, display, Image as IImage
import time

# Initialize clients
REGION = boto3.session.Session().region_name
rekognition = boto3.client('rekognition', REGION)
s3 = boto3.client('s3', REGION)

%store -r bucket_name

# Detect objects in image
***

In [ ]:
# Show image
image_name = "media/cellphone.jpg"
display(IImage(url=s3.generate_presigned_url('get_object', Params={'Bucket': bucket_name, 'Key': image_name})))

#### Call Rekognition to detect objects in the image

Call Amazon Rekognition to detect objects in the image
<https://docs.aws.amazon.com/rekognition/latest/dg/API_DetectLabels.html>

In [ ]:
detect_labels_response = rekognition.detect_labels(
    Image={
        'S3Object': {
            'Bucket': bucket_name,
            'Name': image_name,
        }
    }
)

#### Review the raw JSON reponse from Rekognition

In the JSON response below, you will see Label, detected instances, confidence score and additional information.

In [ ]:
display(detect_labels_response)

#### Display list of detected unsafe objects

You can build your own list of flagged objects.

In [ ]:
flagged_objects = ["Cell Phone", "Electronics" "Mobile Phone", "Phone", "Reading"]

for label in detect_labels_response["Labels"]:
    if(label["Name"] in flagged_objects):
        print("Detected unsafe object:")
        print("- {} (Confidence: {})".format(label["Name"], label["Confidence"]))
        print("  - Parents: {}".format(label["Parents"]))

# Recognize objects in video
 Object recognition in video is an async operation. 
https://docs.aws.amazon.com/rekognition/latest/dg/API_StartLabelDetection.html. 

- First we start a label detection job which returns a Job Id.
- We can then call `get_label_detection` to get the job status and after job is complete, we can get object metadata.
- In production use cases, you would usually use StepFunction or SNS topic to get notified when job is complete.
***

#### Show video in the player

In [ ]:
video_name = "media/objects.mp4"
s3_video_url = s3.generate_presigned_url('get_object', Params={'Bucket': bucket_name, 'Key': video_name})

video_tag = "<video controls='controls' autoplay width='640' height='360' name='Video' src='{0}'></video>".format(s3_video_url)
video_ui = "<table><tr><td style='vertical-align: top'>{}</td></tr></table>".format(video_tag)
display(HTML(video_ui))


#### Call Rekognition to start a job for object detection

In [ ]:
start_label_detection = rekognition.start_label_detection(
    Video={
        'S3Object': {
            'Bucket': bucket_name,
            'Name': video_name,
        }
    },
)

labels_job_id = start_label_detection['JobId']
display("Job Id: {0}".format(labels_job_id))

### Additional (Optional) Request Attributes

ClientRequestToken:
https://docs.aws.amazon.com/rekognition/latest/dg/API_StartLabelDetection.html#rekognition-StartLabelDetection-request-ClientRequestToken

JobTag:
https://docs.aws.amazon.com/rekognition/latest/dg/API_StartLabelDetection.html#rekognition-StartLabelDetection-request-JobTag

MinConfidence:
https://docs.aws.amazon.com/rekognition/latest/dg/API_StartLabelDetection.html#rekognition-StartLabelDetection-request-MinConfidence

NotificationChannel:
https://docs.aws.amazon.com/rekognition/latest/dg/API_StartLabelDetection.html#rekognition-StartLabelDetection-request-NotificationChannel


#### Wait for object detection job to complete

In production use cases, you would usually use StepFunction or SNS topic to get notified when job is complete.

In [ ]:
get_object_detection = rekognition.get_label_detection(
    JobId=labels_job_id,
    SortBy='TIMESTAMP'
)

while(get_object_detection['JobStatus'] == 'IN_PROGRESS'):
    time.sleep(5)
    print('.', end='')
 
    get_object_detection = rekognition.get_label_detection(
        JobId=labels_job_id,
        SortBy='TIMESTAMP')
    
display(get_object_detection['JobStatus'])

#### Review raw JSON reponse from Rekognition

In the JSON response below, you will see list of detected objects and activities.
For each detected object, you will see information like Timestamp.

In [ ]:
display(get_object_detection)

#### Display names of recognized objects in the video

In [ ]:
the_objects = {}

# Display timestamps and objects detected at that time
str_detail = "Objects detected in video<br>=======================================<br>"
str_overall = "Objects in the overall video:<br>=======================================<br>"

# Objects detected in each frame
for obj in get_object_detection['Labels']:
    ts = obj ["Timestamp"]
    cconfidence = obj['Label']["Confidence"]
    oname = obj['Label']["Name"]
    
    if(oname in flagged_objects):
        print("Found flagged object at {} ms: {} (Confidence: {})".format(ts, oname, round(cconfidence,2)))
    
    str_detail += "At {} ms: {} (Confidence: {})<br>".format(ts, oname, round(cconfidence,2))
    if oname in the_objects:
        cojb = the_objects[oname]
        the_objects[oname] = {"Name" : oname, "Count": 1+cojb["Count"]}
    else:
        the_objects[oname] = {"Name" : oname, "Count": 1}

# Unique objects detected in video
for the_object in the_objects:
    str_overall += "Name: {}, Count: {}<br>".format(the_object, the_objects[the_object]["Count"])

# Display results
display(HTML(str_overall))

In [ ]:
list_ui = "<table><tr><td style='vertical-align: top'>{}</td></tr></table>".format(str_detail)
display(HTML(list_ui))

***
### References
- https://docs.aws.amazon.com/rekognition/latest/dg/API_DetectLabels.html
- https://docs.aws.amazon.com/rekognition/latest/dg/API_StartLabelDetection.html
- https://docs.aws.amazon.com/rekognition/latest/dg/API_GetLabelDetection.html

***

You have successfully used Amazon Rekognition to identify specific objects in images and videos.